In [1]:
from glom import glom

In [3]:
data = {'a': {'b': {'c': 'd'}}}

In [4]:
# 기존방식으로 출력
print(data['a']['b']['c'])

# glom 으로 출력
print(glom(data, 'a.b.c' , default = None))



d
d


In [5]:
# glom 의 장점은 해당 키값이 없을 때 오류가 아니라 default 값을 출력하기에 관리 하기 매우 좋다

try:
    print(data['a']['b']['d'])
except KeyError:
    print("KeyError 발생")


KeyError 발생


In [6]:
try:
    print(glom(data, 'a.b.d' , default = None))
except KeyError:
    print("KeyError 발생")

None


key error 없이 내 마음대로 값을 뽑아내기 매우 편함

## Going Beyond

output = glom(data, spec)

data 는 분석할 데이터
spec 은 검색한 결과를 우리가 원하는 형식으로 변경하여 만들 수 있다.

In [7]:
target = {
    'galaxy': {
        'system': {
            'planet': 'jupiter'
        }
    }
}

spec = "galaxy.system.planet"

print(glom(target, spec))


jupiter


리스트 구조를 가지고 있는 키 값 하위에 동일값의 키값이 있다면


튜플 형태로 spec 을 구성하면 리스트로 값을 꺼낼수 있다


("공통 키값" , ['리스트내 공통 키'])

("path" , ['subpath'])

In [8]:
# spec 을 내가 원하는 형식으로 재구조화 가능
target = {
    'system': {
        'planets': [
            {'name': 'earth'},
            {'name': 'jupiter'}
        ]
    }
}

print(glom(target, ('system.planets', ['name'])))


['earth', 'jupiter']


In [9]:
target = {
    'system': {
        'planets': [
            {'name': 'earth', 'moons': 1},
            {'name': 'jupiter', 'moons': 69}
        ]
    }
}

spec = {
    "name" : ("system.planets", ["name"]),
    "moons" : ("system.planets", ["moons"])
}

print(glom(target, spec))

{'name': ['earth', 'jupiter'], 'moons': [1, 69]}


In [12]:
target = {
    'system': {
        'planets': [
            {
                'name': 'earth',
                'moons': [
                    {'name': 'luna'}
                ]
            },
            {
                'name': 'jupiter',
                'moons': [
                    {'name': 'io'},
                    {'name': 'europa'}
                ]
            }
        ]
    }
}

spec = {
    "planet_name" : ('system.planets', ['name']),
    "moon_name" : ('system.planets', [('moons', ['name'])])
}

print(glom(target,spec))

{'planet_name': ['earth', 'jupiter'], 'moon_name': [['luna'], ['io', 'europa']]}


In [ ]:
# Coalesce 는 여러개의 키값을 넣어서 찾고 싶은 키값을 찾을 수 있다.

# Coalesce 는 여러개의 키값을 넣어서 찾고 싶은 키값을 찾을 수 있다.

즉 없으면 그냥 넘어갈 수 있는것이다.

동일한 값에 대해서 키 값 업데이트가 이루어졌을 때 사용하면 좋을 듯

In [13]:
from glom import Coalesce

target = {
    'system': {
        'planets': [
            {'name': 'earth', 'moons': 1},
            {'name': 'jupiter', 'moons': 69}
        ]
    }
}

spec = {
    "planets" : (Coalesce('system.planets', 'system.dwarf_planets'), ['name']),
    "moons" : (Coalesce('system.planets', 'system.dwarf_planets'), ['moons'])
}

print(glom(target, spec))

{'planets': ['earth', 'jupiter'], 'moons': [1, 69]}


In [14]:
target = {
    'system': {
        'dwarf_planets': [
            {'name': 'pluto', 'moons': 5},
            {'name': 'ceres', 'moons': 0}
        ]
    }
}

print(glom(target, spec))

{'planets': ['pluto', 'ceres'], 'moons': [5, 0]}


In [16]:
from glom import glom, T, Merge, Iter, Coalesce

target = {
   "pluto": {"moons": 6, "population": None},
   "venus": {"population": {"aliens": 5}},
   "earth": {"moons": 1, "population": {"humans": 7700000000, "aliens": 1}},
}

spec = {
    "moons": (
         T.items(),
         Iter({T[0]: (T[1], Coalesce("moons", default=0))}),
         Merge(),
    )
}

print(glom(target, spec))

{'moons': {'pluto': 6, 'venus': 0, 'earth': 1}}


Python 기본 함수를 사용가능

In [17]:
target = {
    'system': {
        'planets': [
            {'name': 'earth', 'moons': 1},
            {'name': 'jupiter', 'moons': 69}
        ]
    }
}

print(glom(target, {'moon_count': ('system.planets', ['moons'], sum)}))

{'moon_count': 70}
